In [2]:
#Add needed imports
import numpy as np
import pandas as pd
import os
import shap
from numpy import mean
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras 
import pydot
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from keras.utils import plot_model
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import os
import shap

#Read data
proccessed_data_path =os.path.join(os.path.pardir,os.path.pardir,'data','processed')
train_path = os.path.join(proccessed_data_path,'dataset2.csv')
df = pd.read_csv(train_path)
labels=df['Churn']
x = df.drop(columns=['Churn','Unnamed: 0'],axis = 'columns')
y=np.ravel(labels)
oversample = SMOTENC(categorical_features=[ 1,2,4,5,6,7,8,9,10,11,12,13,14,15,18])
x, y = oversample.fit_resample(x, y)
sc = RobustScaler()
x = pd.DataFrame(sc.fit_transform(x),columns = x.columns)

ii = 1

xgb_model = XGBClassifier(random_state=1,learning_rate=0.05, max_depth=7,eval_metric='mlogloss',use_label_encoder =False ,objective="binary:logistic")
dt_model=DecisionTreeClassifier(random_state=1,criterion='entropy',max_depth = 7,min_samples_leaf=30)  
nn_model = Sequential()
nn_model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.001), input_dim=19, activation='relu' ))
nn_model.add(Dropout(rate=0.2))
nn_model.add(Dense(8,kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu'))
nn_model.add(Dropout(rate=0.1))
nn_model.add(Dense(1, activation='sigmoid'))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1  

Iteration 1


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.096934
                      0.036518
                      0.170941
                      0.331664
                      0.013367
                      0.121877
                      0.121362
                      0.196041
                      0.099288
                      0.033348
                      0.161025
                      0.125109
                      0.061082
                      1.272508
                      0.272760
                      0.280262
                      0.441782
                      0.255159
                      0.050270
 DT - feature_importance_vals
                     0.000000
                     0.000000
                     0.020011
                     0.045050
                     0.000000
                     0.009494
                     0.026998
                     0.036561
                     0.003348
                     0.000000
                     0.038766
                    

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.113893
                      0.039962
                      0.152585
                      0.332550
                      0.012231
                      0.105551
                      0.100119
                      0.176098
                      0.091471
                      0.032541
                      0.173313
                      0.075134
                      0.120247
                      1.262001
                      0.274608
                      0.272397
                      0.431340
                      0.273142
                      0.071931
 DT - feature_importance_vals
                     0.000000
                     0.000000
                     0.017498
                     0.042545
                     0.000000
                     0.006153
                     0.022637
                     0.048539
                     0.006031
                     0.000000
                     0.040433
                    

...

 XGB - feature_importance_vals
                      0.084060
                      0.031444
                      0.119001
                      0.345331
                      0.008641
                      0.106638
                      0.085347
                      0.172543
                      0.092268
                      0.036866
                      0.143049
                      0.097852
                      0.091503
                      1.225944
                      0.240092
                      0.271319
                      0.471170
                      0.267092
                      0.059623
 DT - feature_importance_vals
                     0.001802
                     0.000000
                     0.004124
                     0.054562
                     0.000000
                     0.002739
                     0.014387
                     0.024500
                     0.000722
                     0.002067
                     0.028130
                    

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.094304
                      0.031137
                      0.138380
                      0.346239
                      0.018733
                      0.096313
                      0.104750
                      0.168426
                      0.079476
                      0.037364
                      0.166962
                      0.071895
                      0.114598
                      1.220590
                      0.259144
                      0.267920
                      0.443578
                      0.257370
                      0.061524
 DT - feature_importance_vals
                     0.000000
                     0.000856
                     0.016452
                     0.034889
                     0.000000
                     0.000000
                     0.027783
                     0.031136
                     0.003069
                     0.000861
                     0.028145
                    

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.095723
                      0.023509
                      0.129738
                      0.349410
                      0.015231
                      0.098654
                      0.105112
                      0.168030
                      0.089829
                      0.031138
                      0.153996
                      0.098925
                      0.088625
                      1.227483
                      0.275637
                      0.276116
                      0.433762
                      0.249637
                      0.063415
 DT - feature_importance_vals
                     0.001921
                     0.000857
                     0.010872
                     0.044009
                     0.002832
                     0.006021
                     0.009984
                     0.025690
                     0.004297
                     0.001247
                     0.030364
                    